In [47]:
import torch
from torchvision import transforms
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import mmcv
import matplotlib.pyplot as plt
import os.path as osp
import webdataset as wds
from datasets.formatting import ToDataContainer
from datasets.builder import WordAugTokenizeWrapper, Tokenize
from datasets.tokenizer import SimpleTokenizer
from mmcv.parallel import collate
from functools import partial



In [54]:
transform = transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
                ToDataContainer()
            ])


text_transform = WordAugTokenizeWrapper(
                Tokenize(SimpleTokenizer(), max_seq_len=77),
                max_word=3,
                max_key=0,
                word_type=['noun'])

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ToDataContainer
)


In [52]:
path = "/workspace/Dataset/local_data/gcc3m_shards/"
prefix = "gcc-train-000000.tar"
tar_file = osp.join(path, prefix)

dataset = (
    wds.WebDataset(tar_file, repeat=True)
    .decode('pil')
    .rename(image1='jpg;png;jpeg', image2='jpg;png;jpeg', text='text;txt', keep=False,)
    .map_dict(image1=transform, image2=transform, text=text_transform)
)

dc_collate = partial(collate, samples_per_gpu=16)

data_loader = wds.WebLoader(
    dataset.batched(16, dc_collate),
    batch_size = 16,
    shuffle=False,
)


In [53]:
for i, data in enumerate(data_loader):
    print(data)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'mmcv.parallel.data_container.DataContainer'>